In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

# Cofiguraciones
%matplotlib inline
plt.rcParams['figure.figsize'] = (10,10)

# Importación de los datos

In [2]:
vehiculos = pd.read_csv("./Data/vehiculos_limpio_analisis.csv")
vehiculos.head()

,fabricante,modelo,year,desplazamiento,cilindros,transmision,traccion,clase,combustible,consumo,co2
0,AM General,DJ Po Vehicle 2WD,1984,2.5,4.0,Automatic 3-spd,2-Wheel Drive,Special Purpose Vehicle 2WD,Regular,17,522.764706
1,AM General,FJ8c Post Office,1984,4.2,6.0,Automatic 3-spd,2-Wheel Drive,Special Purpose Vehicle 2WD,Regular,13,683.615385
2,AM General,Post Office DJ5 2WD,1985,2.5,4.0,Automatic 3-spd,Rear-Wheel Drive,Special Purpose Vehicle 2WD,Regular,16,555.437500
3,AM General,Post Office DJ8 2WD,1985,4.2,6.0,Automatic 3-spd,Rear-Wheel Drive,Special Purpose Vehicle 2WD,Regular,13,683.615385
4,ASC Incorporated,GNX,1987,3.8,6.0,Automatic 4-spd,Rear-Wheel Drive,Midsize Cars,Premium,16,555.437500


# Análisis de agrupación

Lo primero que debemos hacer es calcular el número de valores únicos para cada campo

In [5]:
def unique_col_values(df):
    for col in df:
        print("{} | {} | {}".format(
            col,
            len(df[col].unique()),
            df[col].dtype
        ))

unique_col_values(vehiculos)

fabricante | 129 | object
modelo | 3734 | object
year | 35 | int64
desplazamiento | 66 | float64
cilindros | 10 | float64
transmision | 38 | object
traccion | 8 | object
clase | 34 | object
combustible | 13 | object
consumo | 49 | int64
co2 | 596 | float64


# Agrupación variables categóricas

Exceptuando el fabricante y el modelo, ya que estas variables aunque tienen una alta cardinalidad no se pueden agrupar debido a que cada modelo y fabricante es diferente por algo, las demás variables categóricas (transmisión, tracción, clase, combustible) sí que se pueden agrupar.

## Transmisión

In [7]:
vehiculos.transmision.unique()

array(['Automatic 3-spd', 'Automatic 4-spd', 'Manual 5-spd',
       'Automatic (S5)', 'Manual 6-spd', 'Automatic 5-spd',
       'Automatic (AM8)', 'Automatic (AM-S8)', 'Automatic (AV-S7)',
       'Automatic (S6)', 'Automatic (S9)', 'Automatic (AM-S7)',
       'Automatic (S4)', 'Automatic (AM-S9)', 'Automatic (S7)',
       'Automatic (AM7)', 'Automatic (AM6)', 'Automatic 6-spd',
       'Automatic 8-spd', 'Manual 4-spd', 'Automatic (S8)',
       'Manual 7-spd', 'Automatic (AM-S6)', 'Auto(AM-S6)',
       'Automatic (variable gear ratios)', 'Automatic (AV-S8)',
       'Automatic (A1)', 'Automatic (AV-S6)', 'Manual 3-spd',
       'Automatic (S10)', 'Automatic 9-spd', 'Manual 4-spd Doubled', nan,
       'Automatic (L4)', 'Automatic (L3)', 'Automatic (AV-S10)',
       'Automatic 7-spd', 'Automatic (AM5)'], dtype=object)

In [26]:
vehiculos['transmision_tipo'] = "Automatica"

vehiculos.loc[
    vehiculos.transmision.notnull() & vehiculos.transmision.str.startswith('M')
    , "transmision_tipo"] = "Manual"

Pandas tiene un dtype especial para variables categoricas llamado category. El especificar a pandas que un campo es una categoria en vez de un object (un string generalmente) amplia las funcionalidades que podemos usar.Además, las categorias en general ocupan menos espacio en memoria (si su cardinalidad no es muy elevada).

In [27]:
vehiculos.transmision_tipo = vehiculos.transmision_tipo.astype("category")
vehiculos.transmision_tipo.value_counts()

Automatica    24937
Manual        11854
Name: transmision_tipo, dtype: int64

## Tracción

In [28]:
vehiculos.traccion.unique()

array(['2-Wheel Drive', 'Rear-Wheel Drive', 'Front-Wheel Drive',
       '4-Wheel or All-Wheel Drive', 'All-Wheel Drive', nan,
       '4-Wheel Drive', 'Part-time 4-Wheel Drive'], dtype=object)

In [29]:
vehiculos["traccion_tipo"] = "dos"

vehiculos.loc[vehiculos.traccion.isin([
    "4-Wheel or All-Wheel Drive", "All-Wheel Drive",
    "4-Wheel Drive", "Part-time 4-Wheel Drive"
]), "traccion_tipo"] = "cuatro"

In [30]:
vehiculos.traccion_tipo = vehiculos.traccion_tipo.astype("category")
vehiculos.traccion_tipo.value_counts()

dos       26868
cuatro     9923
Name: traccion_tipo, dtype: int64

## Clase

In [31]:
vehiculos.clase.unique()

array(['Special Purpose Vehicle 2WD', 'Midsize Cars', 'Subcompact Cars',
       'Compact Cars', 'Sport Utility Vehicle - 4WD',
       'Small Sport Utility Vehicle 2WD',
       'Small Sport Utility Vehicle 4WD', 'Two Seaters',
       'Sport Utility Vehicle - 2WD', 'Special Purpose Vehicles',
       'Special Purpose Vehicle 4WD', 'Small Station Wagons',
       'Minicompact Cars', 'Midsize-Large Station Wagons',
       'Midsize Station Wagons', 'Large Cars',
       'Standard Sport Utility Vehicle 4WD',
       'Standard Sport Utility Vehicle 2WD', 'Minivan - 4WD',
       'Minivan - 2WD', 'Vans', 'Vans, Cargo Type',
       'Vans, Passenger Type', 'Standard Pickup Trucks 2WD',
       'Standard Pickup Trucks', 'Standard Pickup Trucks/2wd',
       'Small Pickup Trucks 2WD', 'Standard Pickup Trucks 4WD',
       'Small Pickup Trucks 4WD', 'Small Pickup Trucks', 'Vans Passenger',
       'Special Purpose Vehicle', 'Special Purpose Vehicles/2wd',
       'Special Purpose Vehicles/4wd'], dtype=object

In [50]:
pequeno = ['Compact Cars','Subcompact Cars','Two Seaters','Minicompact Cars']
medio = ['Midsize Cars']
grande = ['Large Cars']

vehiculos.loc[vehiculos['clase'].isin(pequeno), 
             'clase_tipo'] = 'Coche Pequeño'

vehiculos.loc[vehiculos['clase'].isin(medio), 
             'clase_tipo'] = 'Coche Mediano'

vehiculos.loc[vehiculos['clase'].isin(grande), 
             'clase_tipo'] = 'Coche Grande'

vehiculos.loc[vehiculos['clase'].str.contains('Truck'), 
             'clase_tipo'] = 'Camioneta'

vehiculos.loc[vehiculos['clase'].str.contains('Special Purpose'), 
             'clase_tipo'] = 'Vehículo Especial'

vehiculos.loc[vehiculos['clase'].str.contains('Sport Utility'), 
             'clase_tipo'] = 'Deportivo'

vehiculos.loc[vehiculos['clase'].str.contains('Station'), 
             'clase_tipo'] = 'Coche Familiar'

vehiculos.loc[(vehiculos['clase'].str.lower().str.contains('van')),
             'clase_tipo'] = 'Furgoneta'

In [51]:
vehiculos.clase_tipo = vehiculos.clase_tipo.astype("category")
vehiculos.clase_tipo.value_counts()

Coche Pequeño        13007
Camioneta             5439
Deportivo             5289
Coche Mediano         4261
Coche Familiar        2533
Vehículo Especial     2214
Furgoneta             2211
Coche Grande          1837
Name: clase_tipo, dtype: int64

## Combustible

In [55]:
vehiculos.combustible.unique()

array(['Regular', 'Premium', 'Diesel', 'Premium and Electricity',
       'Premium or E85', 'Premium Gas or Electricity', 'Gasoline or E85',
       'Gasoline or natural gas', 'CNG', 'Regular Gas or Electricity',
       'Midgrade', 'Regular Gas and Electricity', 'Gasoline or propane'],
      dtype=object)

In [56]:
vehiculos['combustible_tipo'] = 'Otros tipos de combustible'

vehiculos.loc[vehiculos['combustible']=='Regular',
             'combustible_tipo'] = 'Normal'

vehiculos.loc[vehiculos['combustible']=='Premium',
             'combustible_tipo'] = 'Premium'

vehiculos.loc[vehiculos['combustible'].str.contains('Electricity'),
             'combustible_tipo'] = 'Hibrido'

In [57]:
vehiculos.combustible_tipo = vehiculos.combustible_tipo.astype("category")
vehiculos.combustible_tipo.value_counts()

Normal                        24078
Premium                       10206
Otros tipos de combustible     2437
Hibrido                          70
Name: combustible_tipo, dtype: int64

## Visualizamos la tabla

Una vez realizado el proceso de agrupar las variables categóricas, comprobamos que las nuevas agrupaciones son correctas.

In [58]:
vehiculos.head()

,fabricante,modelo,year,desplazamiento,cilindros,transmision,traccion,clase,combustible,consumo,co2,transmision_tipo,traccion_tipo,clase_tipo,combustible_tipo
0,AM General,DJ Po Vehicle 2WD,1984,2.5,4.0,Automatic 3-spd,2-Wheel Drive,Special Purpose Vehicle 2WD,Regular,17,522.764706,Automatica,dos,Vehículo Especial,Normal
1,AM General,FJ8c Post Office,1984,4.2,6.0,Automatic 3-spd,2-Wheel Drive,Special Purpose Vehicle 2WD,Regular,13,683.615385,Automatica,dos,Vehículo Especial,Normal
2,AM General,Post Office DJ5 2WD,1985,2.5,4.0,Automatic 3-spd,Rear-Wheel Drive,Special Purpose Vehicle 2WD,Regular,16,555.437500,Automatica,dos,Vehículo Especial,Normal
3,AM General,Post Office DJ8 2WD,1985,4.2,6.0,Automatic 3-spd,Rear-Wheel Drive,Special Purpose Vehicle 2WD,Regular,13,683.615385,Automatica,dos,Vehículo Especial,Normal
4,ASC Incorporated,GNX,1987,3.8,6.0,Automatic 4-spd,Rear-Wheel Drive,Midsize Cars,Premium,16,555.437500,Automatica,dos,Coche Mediano,Premium


# Agrupación variables cuantitativas

Para poder agrupar variables númericas, lo más fácil es usar cuartiles/quintiles, éstos nos dividen la muestra en 4/5 grupos. Por otro lado, solo tiene sentido agrupar las variables cuantitativas continuas, ya que las discretas solo pueden tomar valores finitos. Las variables a estudiar son: desplazamiento, consumo y co2. Tanto fecha como cilindros no tiene sentido agruparlas, ya que toman valores finitos.

## Desplazamiento

In [60]:
tipos_tam_motor = ['muy pequeño', "pequeño", "mediano", "grande", "muy grande"]

vehiculos['tamano_motor_tipo'] = pd.qcut(vehiculos['desplazamiento'],
                                  5, tipos_tam_motor)

In [61]:
vehiculos.tamano_motor_tipo.value_counts()

muy pequeño    8245
mediano        7892
muy grande     7002
grande         6997
pequeño        6653
Name: tamano_motor_tipo, dtype: int64

## Consumo

In [62]:
tipos_consumo = ['muy bajo', 'bajo', 'moderado', 'alto', 'muy alto']

vehiculos['consumo_tipo'] = pd.qcut(vehiculos['consumo'],
                                  5, tipos_consumo)

In [63]:
vehiculos.consumo_tipo.value_counts()

moderado    9760
muy bajo    9151
muy alto    6013
bajo        6003
alto        5864
Name: consumo_tipo, dtype: int64

## CO2

In [64]:
tipos_co2 = ['muy bajo', 'bajo', 'moderado', 'alto', 'muy alto']

vehiculos['co2_tipo'] = pd.qcut(vehiculos['co2'],
                                  5, tipos_co2)

In [65]:
vehiculos.co2_tipo.value_counts()

moderado    10118
bajo         7380
muy bajo     7379
muy alto     7152
alto         4762
Name: co2_tipo, dtype: int64

## Consumo litros por milla

En la descripción del dataset, se ve que CO2 se mide en gramos por milla, mientras que el consumo se mide en millas por galon.

Lo que debemos hacer es convertir el consumo a galones por milla y asi ambas variables son relativas a la milla y podemos compararlas.

In [66]:
litros_por_galon =  3.78541

vehiculos["consumo_litros_milla"] = litros_por_galon/ vehiculos.consumo

In [67]:
tipos_consumo = ['muy bajo', 'bajo', 'moderado', 'alto', 'muy alto']

vehiculos['consumo_tipo'] = pd.qcut(vehiculos['consumo_litros_milla'],
                                  5, labels=tipos_consumo)

## Visualizamos la tabla

Una vez realizado el proceso de agrupar las variables continuas, comprobamos que las nuevas agrupaciones son correctas.

In [68]:
vehiculos.head()

,fabricante,modelo,year,desplazamiento,cilindros,transmision,traccion,clase,combustible,consumo,co2,transmision_tipo,traccion_tipo,clase_tipo,combustible_tipo,tamano_motor_tipo,consumo_tipo,co2_tipo,consumo_litros_milla
0,AM General,DJ Po Vehicle 2WD,1984,2.5,4.0,Automatic 3-spd,2-Wheel Drive,Special Purpose Vehicle 2WD,Regular,17,522.764706,Automatica,dos,Vehículo Especial,Normal,pequeño,alto,alto,0.222671
1,AM General,FJ8c Post Office,1984,4.2,6.0,Automatic 3-spd,2-Wheel Drive,Special Purpose Vehicle 2WD,Regular,13,683.615385,Automatica,dos,Vehículo Especial,Normal,grande,muy alto,muy alto,0.291185
2,AM General,Post Office DJ5 2WD,1985,2.5,4.0,Automatic 3-spd,Rear-Wheel Drive,Special Purpose Vehicle 2WD,Regular,16,555.437500,Automatica,dos,Vehículo Especial,Normal,pequeño,alto,alto,0.236588
3,AM General,Post Office DJ8 2WD,1985,4.2,6.0,Automatic 3-spd,Rear-Wheel Drive,Special Purpose Vehicle 2WD,Regular,13,683.615385,Automatica,dos,Vehículo Especial,Normal,grande,muy alto,muy alto,0.291185
4,ASC Incorporated,GNX,1987,3.8,6.0,Automatic 4-spd,Rear-Wheel Drive,Midsize Cars,Premium,16,555.437500,Automatica,dos,Coche Mediano,Premium,grande,alto,alto,0.236588


In [69]:
vehiculos.dtypes

fabricante                object
modelo                    object
year                       int64
desplazamiento           float64
cilindros                float64
transmision               object
traccion                  object
clase                     object
combustible               object
consumo                    int64
co2                      float64
transmision_tipo        category
traccion_tipo           category
clase_tipo              category
combustible_tipo        category
tamano_motor_tipo       category
consumo_tipo            category
co2_tipo                category
consumo_litros_milla     float64
dtype: object

# Conclusiones

- La variable consumo esta definida en millas por galon y la variable co2 está definida como gramos por milla. Dado que el co2 es la variable principal del dataset, hemos creado la variable consumo_litros_milla definida como litros por milla para poder comparar con co2.

# Exportación

In [72]:
vehiculos.to_pickle("./Data/vehiculos_variables_agrupadas.pkl")